In [50]:
# import des librairies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mysql.connector

In [51]:
# Lecture du fichier csv
df = pd.read_csv('../data/fr-esr-insertion_professionnelle-master.csv', delimiter=';')

In [52]:
# Connecter à la base de données
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Goodboy.5",
  database="insertion_pro",
  auth_plugin='mysql_native_password'
)

In [53]:
# Remove all the rows where the column "remarque" is not empty
df = df[df['remarque'].isnull()]

In [54]:
## Insert values to Academie table
# Fetch the needed columns
id_academie = df['code_de_l_academie']
nom_academie = df['academie']

# Delete all duplicates id academies using panda's drop_duplicates function
id_academie = id_academie.drop_duplicates()
nom_academie = nom_academie.drop_duplicates()

# Delete null values
id_academie = id_academie.dropna()
nom_academie = nom_academie.dropna()

# Fetch only values
id_academie = id_academie.values
nom_academie = nom_academie.values

In [55]:
# Insert values to Academie table
for i in range(len(id_academie)):
    sql = "INSERT INTO academie (id_academie, nom_academie) VALUES (%s, %s)"
    val = (id_academie[i], nom_academie[i])
    mycursor = mydb.cursor()
    mycursor.execute(sql, val)
    mydb.commit()

In [56]:
## Insert values to Etablisement table
# Fetch the needed columns
# Delete all rows where numero_d_etablissement is duplicated
df_id_etablissement = df.drop_duplicates(subset='numero_de_l_etablissement')

# Delete all rows where id_academie is null
df_id_etablissement = df_id_etablissement[df_id_etablissement['code_de_l_academie'].notnull()]

# Replace all null values by None
df_id_etablissement = df_id_etablissement.where((pd.notnull(df_id_etablissement)), None)

# Fetch the needed columns
id_etablissement = df_id_etablissement['numero_de_l_etablissement'].values
id_academie = df_id_etablissement['code_de_l_academie'].values
nom_etablissement = df_id_etablissement['etablissement'].values
nom_etablissement_actuel = df_id_etablissement['etablissementactuel'].values

In [57]:
# Insert values to Etablissement table
# If value is nan then insert null
# Check if the value is already in the table
for i in range(len(id_etablissement)):
    sql = "SELECT * FROM etablissement WHERE id_etablissement = %s"
    val = (id_etablissement[i],)
    mycursor = mydb.cursor()
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    if len(myresult) == 0:
        sql = "INSERT INTO etablissement (id_etablissement, id_academie, nom_etablissement, nom_etablissement_actuel) VALUES (%s, %s, %s, %s)"
        val = (id_etablissement[i], id_academie[i], nom_etablissement[i], nom_etablissement_actuel[i])
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)
        mydb.commit()

In [58]:
## Insert values to Domaine table
# Fetch the needed columns
code_domaine = df['code_du_domaine']
nom_domaine = df['domaine']

# Delete all duplicates id academies using panda's drop_duplicates function
code_domaine = code_domaine.drop_duplicates()
nom_domaine = nom_domaine.drop_duplicates()

# Delete null values
code_domaine = code_domaine.dropna()
nom_domaine = nom_domaine.dropna()

# Fetch only values
code_domaine = code_domaine.values
nom_domaine = nom_domaine.values


In [59]:
# Insert values to Domaine table
for i in range(len(code_domaine)):
    sql = "INSERT INTO domaine (code_domaine, nom_domaine) VALUES (%s, %s)"
    val = (code_domaine[i], nom_domaine[i])
    mycursor = mydb.cursor()
    mycursor.execute(sql, val)
    mydb.commit()

In [60]:
# Insert values to Discipline table
# Fetch the needed columns
# Delete all rows where code_de_la_discipline is duplicated
df_code_discipline = df.drop_duplicates(subset='code_de_la_discipline')

# Delete all rows where Domaine is null
df_code_discipline = df_code_discipline[df_code_discipline['code_du_domaine'].notnull()]

# Replace all null values by None
df_code_discipline = df_code_discipline.where((pd.notnull(df_code_discipline)), None)

# Fetch the needed columns
id_discipline = df_code_discipline['code_de_la_discipline'].values
code_domaine = df_code_discipline['code_du_domaine'].values
nom_discipline = df_code_discipline['discipline'].values
poids = df_code_discipline['poids_de_la_discipline'].values

In [61]:
# Insert values to Discipline table
# If value is nan then insert null
# Check if the value is already in the table
for i in range(len(id_discipline)):
    sql = "SELECT * FROM discipline WHERE id_discipline = %s"
    val = (id_discipline[i],)
    mycursor = mydb.cursor()
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    if len(myresult) == 0:
        sql = "INSERT INTO discipline (id_discipline, code_domaine, nom_discipline, poids) VALUES (%s, %s, %s, %s)"
        val = (id_discipline[i], code_domaine[i], nom_discipline[i], poids[i])
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)
        mydb.commit()

In [93]:
# Insert values to Enquete table
# Fetch the needed columns
# Delete all rows where annee, situation and diplome are duplicated
df_enquete = df.drop_duplicates(subset=['annee', 'situation', 'diplome'])

# Delete all rows where annee, situation and diplome are null
df_enquete = df_enquete[df_enquete['annee'].notnull()]
df_enquete = df_enquete[df_enquete['situation'].notnull()]
df_enquete = df_enquete[df_enquete['diplome'].notnull()]

# Replace all null values by None
df_enquete = df_enquete.where((pd.notnull(df_enquete)), None)

# Fetch the needed columns
annee = df_enquete['annee'].values
situation = df_enquete['situation'].values
diplome = df_enquete['diplome'].values

In [94]:
# Convert to list
annee = annee.tolist()
situation = situation.tolist()
diplome = diplome.tolist()

In [96]:
# Insert values to Enquete table
# If value is nan then insert null
# Check if the value is already in the table
for i in range(len(annee)):
    sql = "SELECT * FROM enquete WHERE annee = %s AND situation = %s AND diplome = %s"
    val = (annee[i], situation[i], diplome[i])
    mycursor = mydb.cursor()
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    if len(myresult) == 0:
        sql = "INSERT INTO enquete (annee, situation, diplome) VALUES (%s, %s, %s)"
        val = (annee[i], situation[i], diplome[i])
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)
        mydb.commit()


In [ ]:
# Insert values to Statistiques table
# Fetch the needed columns

id_etablissement = df['id_etablissement']
id_discipline = df['code_de_la_discipline']
annee = df['annee']
situation = df['situation']
diplome = df['diplome']

taux_dinsertion = df['taux_dinsertion']
emplois_cadre_ou_professions_intermediaires = df['emplois_cadre_ou_professions_intermediaires']
emplois_stables = df['emplois_stables']
emplois_a_temps_plein = df['emplois_a_temps_plein']
salaire_net_median_des_emplois_a_temps_plein = df['salaire_net_median_des_emplois_a_temps_plein']
salaire_brut_annuel_estime = df['salaire_brut_annuel_estime']
de_diplome_boursiers = df['de_diplome_boursiers']
taux_de_chomage_regional = df['taux_de_chomage_regional']
salaire_net_mensuel_median_regional = df['salaire_net_mensuel_median_regional']
emplois_cadre = df['emplois_cadre']
emplois_exterieurs_a_la_region_de_luniversite = df['emplois_exterieurs_a_la_region_de_luniversite']
femmes = df['femmes']

# Fetch the needed columns

